In [1]:
import pandas as pd
import re
from typing import Tuple, Dict, FrozenSet
import logging
from datetime import datetime
import numpy as np

In [2]:
df = pd.read_csv('/root/sandbox/disa/data_cleaned.tsv', sep='\t', header=0)


/tmp/ipykernel_2152577/376297877.py:1: DtypeWarning: Columns (198,199,238,357,371,374,385,387,406,407,408,414,416,418,423,424,426,427,428,429,442,443,444,446,447,466,481,491,492,494,495,498,499,500,501,502,503,504,506,511,512,514,521,525,528,529,530,531,532,533,536,539,540,541,543,550,552,553,554,564,566,572,573,577,578,579,580,585,587,590,591,593,594,595,596,609,611,612,613,614,633,658,659,661,662,665,667,668,669,670,672,673,678,679,681,682,685,686,688,689,692,694,696,697,698,699,700,703,706,707,708,710,717,719,720,731,733,739,740,744,745,746,747,752,754,757,758,760,761,762,763,776,780,781,825,826,828,829,836,837,846,848,850,852,853,855,856,861,865,867,870,873,874,875,877,884,886,887,898,900,906,911,912,913,914,993,995,1003,1004,1019,1020,1023,1028,1034,1224,1225,1232,1238,1253,1255,1290,1294,1298,1301,1311,1313,1320,1321,1322,1323,1621,1639,1640,1641,1654,1676,1694,1695,1696,1709,1719,1721,1722,1731,1759,1769,1776) have mixed types. Specify dtype option on import or set low_memory=Fa

In [5]:
from pipeline.cleanup_h3 import classify_h3_fullyorganic


df = classify_h3_fullyorganic(df)


Skipping row for organic classification: AT-GT-KR-KP-2168, event: visit_1_arm_1 since all values are null
Skipping row for organic classification: AT-GT-KR-KP-2168, event: visit_2_arm_1 since all values are null
Skipping row for organic classification: AT-GT-KR-KP-2168, event: visit_1_arm_2 since all values are null
Skipping row for organic classification: AT-GT-KR-KP-2168, event: visit_2_arm_2 since all values are null
Skipping row for organic classification: AT-GT-KR-KP-2169, event: visit_1_arm_1 since all values are null
Skipping row for organic classification: AT-GT-KR-KP-2169, event: visit_2_arm_1 since all values are null
Skipping row for organic classification: AT-GT-KR-KP-2169, event: visit_1_arm_2 since all values are null
Skipping row for organic classification: AT-GT-KR-KP-2169, event: visit_2_arm_2 since all values are null
Skipping row for organic classification: AT-GT-KR-KP-2170, event: visit_1_arm_1 since all values are null
Skipping row for organic classification: AT-GT

AttributeError: 'DataFrame' object has no attribute 'unique'

In [3]:
newdf = df[["h3_plot1organic_calc", "h3_plot2organic_calc", "h3_plot3organic_calc", "h3_fullyorganic"]]
newdf

,h3_plot1organic_calc,h3_plot2organic_calc,h3_plot3organic_calc,h3_fullyorganic
0,NaN,NaN,NaN,['nan']
1,NaN,NaN,NaN,['nan']
2,Conventional,Organic,Organic,['Partially Organic']
3,NaN,NaN,NaN,['nan']
4,NaN,NaN,NaN,['nan']
...,...,...,...,...
7223,NaN,NaN,NaN,['nan']
7224,NaN,NaN,NaN,['nan']
7225,NPM,Conventional,NPM,['Not Organic']
7226,NaN,NaN,NaN,['nan']


In [51]:
def classify_plotN(row, plot_number):
    synthetic_fert_column_name = f"h3_plot{plot_number}fert___1"
    classification_column_name = f"h3_plot{plot_number}organic_calc"
    fert_other_column_name = f"h3_plot{plot_number}fert_other"
    pesticide_column_name = f"h3_plot{plot_number}pest___1"
    pesticide_other_column_name = f"h3_plot{plot_number}pest___777"
    # TODO: Unsure if the current logic accounts for the other category correctly
    # First eliminate all the conventional
    if (row[synthetic_fert_column_name] == "Checked") and (
        row[pesticide_column_name] == "Checked"
    ):
        row[classification_column_name] = "Conventional"
    elif (
        (row[synthetic_fert_column_name] == "Unchecked")
        and (row[fert_other_column_name] != "yes")
        and (row[pesticide_column_name] != "Checked")  # Don't Care pesticide
    ):  # Adding the extra condition incase its labeled data
        row[classification_column_name] = "Organic"
    elif (
        row[pesticide_column_name] != "Checked"
        or row[pesticide_other_column_name] == "Checked"
    ):  # Don't care logic for fertilizer
        row[classification_column_name] = "NPM"
    else:
        row[classification_column_name] = "Unknown"

    return row


In [52]:
df = df.apply(classify_plotN, plot_number=1, axis=1)

In [55]:
plot_number = 1
dfnew = df[[f"h3_plot{plot_number}organic_calc", f"h3_plot{plot_number}fert___1", f"h3_plot{plot_number}fert_other", f"h3_plot{plot_number}pest___1" ]]

dfnew

,h3_plot1organic_calc,h3_plot1fert___1,h3_plot1fert_other,h3_plot1pest___1
0,NPM,NaN,NaN,NaN
1,NPM,NaN,NaN,NaN
2,Conventional,Checked,NaN,Checked
3,NPM,NaN,NaN,NaN
4,NPM,NaN,NaN,NaN
...,...,...,...,...
7223,NPM,NaN,NaN,NaN
7224,NPM,NaN,NaN,NaN
7225,NPM,Checked,no,Unchecked
7226,NPM,NaN,NaN,NaN


In [ ]:
dfnew.to_csv("organic_classification_validation.csv")